In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ternary
import time
import random
import numpy as np
import os
import cv2
import copy
import warnings
from tqdm import tqdm

#org_path= 'G:/Projects/Python/Evolution_Simulation/'
org_path='/mnt/Personal/Projects/Python/Evolution_Simulation/'

frames_path = org_path+'tax_frames'
video_path = org_path+'tax_plots.mp4'


In [2]:
def blend_colors(color1, color2, color3, frac1, frac2, frac3):
    """
    Blend three hex colors based on given fractions.

    Args:
    - color1, color2, color3: Hex color strings (e.g., "#FF0000" for red).
    - frac1, frac2, frac3: Fractions corresponding to each color.

    Returns:
    - Blended color as a hex string.
    """
    # Convert hex color to RGB tuple
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    # Convert RGB tuple to hex color
    def rgb_to_hex(rgb_color):
        return '#{:02x}{:02x}{:02x}'.format(*rgb_color)

    # Get RGB values for each color
    rgb1 = hex_to_rgb(color1)
    rgb2 = hex_to_rgb(color2)
    rgb3 = hex_to_rgb(color3)

    # Calculate blended RGB values
    blended_rgb = (
        int(frac1 * rgb1[0] + frac2 * rgb2[0] + frac3 * rgb3[0]),
        int(frac1 * rgb1[1] + frac2 * rgb2[1] + frac3 * rgb3[1]),
        int(frac1 * rgb1[2] + frac2 * rgb2[2] + frac3 * rgb3[2])
    )

    # Convert blended RGB values back to hex
    blended_hex = rgb_to_hex(blended_rgb)
    return blended_hex

In [3]:

# def plot_specific(data, labels):
#     """
#     Create a ternary plot that traces the values from a list of tuples.

#     Args:
#     - data: List of tuples where each tuple represents a point in ternary space (A, B, C).
#     - labels: List of three labels corresponding to the proportions.
#     """

#     def invert_hex(hex_color):
#         hex_color = hex_color.lstrip('#')
#         rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
#         inverted_rgb = tuple(255 - value for value in rgb)
#         inverted_hex = '#{:02x}{:02x}{:02x}'.format(*inverted_rgb)
#         return inverted_hex

#     # Verify the input data format
#     if any(len(point) != 3 for point in data):
#         raise ValueError("Each point should be a tuple of three values (A, B, C)")

#     # Verify that each point sums to 1
#     for point in data:
#         if round(sum(point), 5) != 1:
#             raise ValueError("Each point should sum to 1")

#     # Set up the ternary plot
#     fig, tax = ternary.figure(scale=1.0)
#     #tax.set_background_color(color="black", alpha=0.7) # the detault, essentially
#     #fig.set_facecolor('black')
#     #tax.boundary(linewidth=2.0)
#     tax.boundary(linewidth=2.0, axes_colors={'l':invert_hex('#ffffff'),'r':invert_hex('#ffffff'),'b':invert_hex('#ffffff')})
#     tax.clear_matplotlib_ticks()
#     #tax.get_axes().patch.set_facecolor('red')
#     tax.get_axes().axis('off')  # Turn off the x and y axis
#     tax.set_title("Ternary Plot", fontsize=24,fontweight='bold')


#     # Set axis labels
#     # tax.left_axis_label(labels[0], fontsize=15, color=invert_hex('#ff0000'))
#     # tax.right_axis_label(labels[1], fontsize=15, color=invert_hex('#0000ff'))
#     # tax.bottom_axis_label(labels[2], fontsize=15, color=invert_hex('#00ff00'))

#     # Convert data to a list of tuples
#     points = data

#     # Plot the trace without dots at the junctions
#     tax.plot(points, color='black', linestyle='-', linewidth=1.5)

#     dot_color=blend_colors('#ff0000','#0000ff','#00ff00',points[-1][0],points[-1][1],points[-1][2])
#     dot_color=invert_hex(dot_color)

#     tax.scatter([points[-1]], marker='o', color=dot_color, s=50)

#     last_point = points[-1]

#     #Format the point values to two decimal places
#     formatted_point = f"({last_point[0]:.2f}, {last_point[1]:.2f}, {last_point[2]:.2f})"

#     tax.annotate(formatted_point, points[-1], fontsize=8, xytext=(0, -10), textcoords='offset points', ha='center')

#     frame_number=len(data)-2

#     frame_filename = f"{frames_path}/frame_{frame_number:03d}.png"
#     plt.savefig(frame_filename, bbox_inches='tight')
#     plt.close()

#     # Show the plot
#     #tax.show()

# def plot_frames(data,labels):
#     for i in tqdm(range(2,len(data)+1)):
#         plot_specific(data[:i],labels)

# def generate_video(data,labels,fps=30):

#     def inverse_image(im):
#         return (255-im)

#     plot_frames(data,labels)

#     # Get the list of frames
#     frame_files = sorted([f for f in os.listdir(frames_path) if f.endswith('.png')])
#     frame_files=frame_files[:len(data)-1]
    
#     img_path = os.path.join(frames_path, frame_files[-1])
#     img = cv2.imread(img_path)
#     height, width, _ = img.shape

#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # for MP4
#     out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

#     # Write all frames to the video
#     for file in frame_files:
#         img_path = os.path.join(frames_path, file)
#         frame = cv2.imread(img_path)
#         frame=inverse_image(frame)
#         out.write(frame)

#     # Release the VideoWriter
#     out.release()
#     print(f"Video successfully created at {video_path}")



In [4]:


def invert_hex(hex_color):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    inverted_rgb = tuple(255 - value for value in rgb)
    inverted_hex = '#{:02x}{:02x}{:02x}'.format(*inverted_rgb)
    return inverted_hex

def plot_specific(data,max_plots):
    """
    Create a ternary plot that traces the values from a list of tuples.

    Args:
    - data: List of tuples where each tuple represents a point in ternary space (A, B, C).
    - labels: List of three labels corresponding to the proportions.
    """

    # Verify the input data format
    if any(len(point) != 3 for point in data):
        raise ValueError("Each point should be a tuple of three values (A, B, C)")

    # Verify that each point sums to 1
    for point in data:
        if round(sum(point), 5) != 1:
            raise ValueError("Each point should sum to 1")

    # Set up the ternary plot
    fig, tax = ternary.figure(scale=1.0)
    #tax.set_background_color(color="black", alpha=0.7) # the detault, essentially
    #fig.set_facecolor('black')
    #tax.boundary(linewidth=2.0)
    tax.boundary(linewidth=5.0, axes_colors={'l':invert_hex('#777777'),'r':invert_hex('#777777'),'b':invert_hex('#777777')})
    tax.clear_matplotlib_ticks()
    #tax.get_axes().patch.set_facecolor('red')
    tax.get_axes().axis('off')  # Turn off the x and y axis
    tax.set_title("Dominance Plot", fontsize=24,fontweight='bold')

    plt.rcParams['lines.antialiased'] = True


    # Set axis labels
    # tax.left_axis_label(labels[0], fontsize=15, color=invert_hex('#ff0000'))
    # tax.right_axis_label(labels[1], fontsize=15, color=invert_hex('#0000ff'))
    # tax.bottom_axis_label(labels[2], fontsize=15, color=invert_hex('#00ff00'))

    # Convert data to a list of tuples
    points = data

    # Plot the trace without dots at the junctions
    tax.plot(points[-max_plots:], color='black', linestyle='-', linewidth=1.3)

    dot_color=blend_colors('#ff0000','#0000ff','#00ff00',points[-1][0],points[-1][1],points[-1][2])
    dot_color=invert_hex(dot_color)
    
    with warnings.catch_warnings(action="ignore"):
        tax.scatter([points[-1]], marker='o', color=dot_color, s=50)

    #last_point = points[-1]

    #Format the point values to two decimal places
    #formatted_point = f"({last_point[0]:.2f}, {last_point[1]:.2f}, {last_point[2]:.2f})"

    #tax.annotate(formatted_point, points[-1], fontsize=8, xytext=(0, -10), textcoords='offset points', ha='center')

    return fig,tax

def plot_with_rectangle(data, labels,max_plots):
    """
    Create a ternary plot that traces the values from a list of tuples and adds a dot with text at the end,
    along with a vertical bar graph beside it.

    Args:
    - data: List of tuples where each tuple represents a point in ternary space (A, B, C).
    - labels: List of three labels corresponding to the proportions.
    - bar_data: List of values for the bar graph.
    - bar_labels: List of labels for the bar graph.
    - bar_colors: List of colors for each bar in the graph.
    """
    # Set up the figure with GridSpec
    #fig = plt.figure(figsize=(12, 6))
    gs = plt.GridSpec(1, 2, width_ratios=[18, 2])

    # Ternary plot
    fig, tax = plot_specific(data,max_plots)
    tax.ax.set_position(gs[0].get_position(fig))
    tax.ax.set_subplotspec(gs[0])

    ratios = data[-1]
    bar_colors = [invert_hex('#ff0000'), invert_hex('#0000ff'), invert_hex('#00ff00')]

    # Draw a rectangle for each ratio
    x = 0.86  # X position of the rectangle (adjust as needed)
    width = 0.14  # Width of the rectangle (adjust as needed)
    bottom = 0.5  # Starting position of the rectangle
    for i, ratio in enumerate(ratios):
        height = ratio * 0.45  # Scale the height for visualization
        rect = Rectangle((x, bottom), width, height, color=bar_colors[i], alpha=0.7)
        fig.gca().add_patch(rect)

        text_x = x + width / 2
        text_y = bottom + height / 2

        if (ratio>=0.07):
            text = f"{labels[i]}"
            #print(text)
            plt.text(text_x, text_y, text, ha='center', va='center', color='black', fontsize=8, fontweight='bold')


        bottom += height

    # Set limits and labels for the rectangle
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xticks([1.03,-0.03])
    plt.yticks([-0.1])

    frame_number=len(data)-2

    frame_filename = f"{frames_path}/frame_{frame_number:03d}.png"
    plt.savefig(frame_filename, bbox_inches='tight',dpi=300)
    plt.close()


def plot_frames(data,labels,max_plots):
    for i in tqdm(range(2,len(data)+1)):
        plot_with_rectangle(data[:i],labels,max_plots)

def generate_video(data,labels,max_plots,fps=30):

    def inverse_image(im):
        return (255-im)

    plot_frames(data,labels,max_plots)

    # Get the list of frames
    frame_files = sorted([f for f in os.listdir(frames_path) if f.endswith('.png')])
    frame_files=frame_files[:len(data)-1]
    
    img_path = os.path.join(frames_path, frame_files[-1])
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # for MP4
    out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

    # Write all frames to the video
    for file in frame_files:
        img_path = os.path.join(frames_path, file)
        frame = cv2.imread(img_path)
        frame=inverse_image(frame)
        out.write(frame)

    # Release the VideoWriter
    out.release()
    print(f"Video successfully created at {video_path}")



In [5]:

# def generate_random_tuples(length):
#     data = []
#     for _ in range(length):
#         # Generate three random numbers
#         x = random.uniform(0, 1)
#         y = random.uniform(0, 1 - x)
#         z = 1 - x - y
#         data.append((x, y, z))
#     return data

# data=generate_random_tuples(100)

# labels = ["Rock", "Scissor", "Paper"]  # The labels for the three variables
# generate_video(data, labels,fps=15)

In [6]:
def count_digits_after_decimal(number):
    # Convert the number to a string
    str_number = str(number)
    
    # Find the position of the decimal point
    if '.' in str_number:
        decimal_position = str_number.index('.')
        # Count the number of digits after the decimal point
        return len(str_number) - decimal_position - 1
    else:
        return 0  # No decimal point means no digits after the decimal


In [7]:

def choosen_eater(state):
    barrier=[state[0],state[0]+state[1]]
    gone=0

    choose=random.randint(1,sum(state))

    if choose<=barrier[0]:
        gone=0
    elif choose<=barrier[1]:
        gone=1
    else:
        gone=2

    state[gone]-=1

    return state,gone

In [8]:
def mutation_chance(born,mutation,mut_max):

    def generate(state):
        pop=copy.deepcopy(state)

        if sum(pop)==0:
            return pop
        
        pos=0

        for i,p in enumerate(pop):
            if p>0:
                pos=i
                break

        for _ in range(pop[pos]):

            rand=random.randint(1,mut_max)

            if rand<=mutation:
                ch=random.randint(1,2)
                pop[pos]-=1
                pop[(pos+ch)%3]+=1

        return pop

    r=born[0]
    s=born[1]
    p=born[2]

    r_rand=generate([r,0,0])
    s_rand=generate([0,s,0])
    p_rand=generate([0,0,p])

    mut=[sum(values) for values in zip(r_rand, s_rand, p_rand)]

    return mut

In [9]:

def children_chance(born):
    perfect=int(born)
    chance=born-int(born)

    if chance==0:
        return perfect

    ch_max=10**(count_digits_after_decimal(chance))
    ch=chance*ch_max

    alive=random.randint(1,ch_max)<=ch

    if alive:
        perfect+=1

    return perfect


In [10]:
def simulate_days(initial_state=[1,1,1],days=100,trees=1000,mango_matrix=None,mutation=0):

    mut_max=10**(count_digits_after_decimal(mutation))
    mutation*=mut_max
    
    if mango_matrix==None:
        mango_matrix= [
            [1, 2, 0],
            [0, 1, 2],
            [2, 0, 1]
        ]

    population=[initial_state]
    #print(population)

    for _ in tqdm(range(days)):
        state=copy.deepcopy(population[-1])
        tree_list=[[0,0,0] for _ in range(trees)]

        for i in range(len(tree_list)):

            if sum(state) < 2:
                break

            state,eat1=choosen_eater(state)
            state,eat2=choosen_eater(state)
            #print(population[-1]," ",state," ",eat1," ",eat2)

            tree=[0,0,0]

            born1=mango_matrix[eat1][eat2] 
            born2=mango_matrix[eat2][eat1]

            born1=children_chance(born1)
            born2=children_chance(born2)

            tree[eat1]+=born1
            tree[eat2]+=born2

            tree=mutation_chance(tree,mutation,mut_max)

            #print(tree)



            tree_list[i]=tree

        new_state=[0,0,0]
        
        for row in tree_list:
            for i in range(3):
                new_state[i] += row[i]

        population.append(new_state)

    return population



In [11]:
# rr rs rp
# sr ss sp
# pr ps pp

win = 2.5
draw = 3.6
loose = 0.5

mango_matrix= [
            [draw, win, loose],
            [loose, draw, win],
            [win, loose, draw]
        ]

mutation=0.00002

days=200
initial_state=[100,100,100]
trees=300000

population = simulate_days(mango_matrix=mango_matrix,initial_state=initial_state,days=days,trees=trees,mutation=mutation)

print(population[::10])

100%|██████████| 200/200 [08:09<00:00,  2.45s/it]

[[100, 100, 100], [257394, 317566, 193372], [1993514, 2067, 57721], [2154402, 0, 1863], [2159832, 0, 44], [2160102, 0, 0], [2160374, 0, 0], [2160411, 0, 0], [2160198, 0, 0], [2159559, 0, 0], [2159547, 0, 0], [2159810, 0, 0], [2159391, 0, 0], [2160114, 0, 0], [2159261, 0, 0], [2159831, 0, 0], [2160055, 0, 0], [2159576, 0, 0], [2159506, 0, 0], [2160468, 0, 0], [2160200, 0, 0]]


In [12]:
data=[]

for pop in population:
    tot=sum(pop)
    ratio=[pop[0]/tot,pop[1]/tot,pop[2]/tot]
    data.append(ratio)

print (population[::50])
print(data[-1])
print(len(data))

[[100, 100, 100], [2160102, 0, 0], [2159547, 0, 0], [2159831, 0, 0], [2160200, 0, 0]]
[1.0, 0.0, 0.0]
201


In [13]:
import shutil
import os

shutil.rmtree(frames_path)

os.makedirs(frames_path)


In [14]:
labels = ["Rock", "Paper", "Scissor"]  # The labels for the three variables

# max_plots=len(data)
max_plots=50

generate_video(data, labels,max_plots,fps=10)

100%|██████████| 200/200 [00:13<00:00, 15.33it/s]


Video successfully created at /mnt/Personal/Projects/Python/Evolution_Simulation/tax_plots.mp4
